# Quantitative Value Strategy

## Imports

In [ ]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install python-dotenv
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install xlsxwriter

In [ ]:
from dotenv import load_dotenv
import math
import numpy
import os
import pandas
import requests
from scipy import stats
import xlsxwriter

## Import List of Stocks

In [ ]:
stocks =  pandas.read_csv('sp_500_stocks.csv')
stocks

## Acquiring API Token

In [ ]:
key = os.getenv('ACCESS_KEY')
key

## Making a First API Call

In [ ]:
symbol = 'AAPL'
url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={key}'
data = requests.get(url).json()
data

### Parsing API Call

In [ ]:
price = data['latestPrice']
peRatio = data['peRatio']

## Executing Batch API Call

In [ ]:
def chuncks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [ ]:
symbol_groups = list(chuncks(stocks['Ticker'], 100))
symbol_groups

In [ ]:
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
symbol_strings

In [ ]:
columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']
columns

In [ ]:
final_dataframe = pandas.DataFrame(columns=columns)
final_dataframe

In [ ]:
for symbol_string in symbol_strings:
    url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={key}'
    data = requests.get(url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pandas.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                data[symbol]['quote']['peRatio'],
                'N/A'
            ], index= columns),
            ignore_index=True
        )

final_dataframe

## Removing Glamour Stocks

In [ ]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending=True, inplace=True)
final_dataframe

In [ ]:
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe

In [ ]:
final_dataframe = final_dataframe[:50]
final_dataframe

In [ ]:
final_dataframe.reset_index(inplace=True)
final_dataframe

In [ ]:
final_dataframe.drop('index', axis=1, inplace=True)
final_dataframe

### Calculating Shares to Buy

In [ ]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the Portfolio size')
    try:
        val = float(portfolio_size)
    except:
        print('Try again. Enter the Portfolio size')
        portfolio_size = input('Enter the Portfolio size')

In [ ]:
portfolio_input()
portfolio_size

In [ ]:
position_size = float(portfolio_size) / len(final_dataframe.index)
position_size

In [ ]:
for i in final_dataframe.index:
    price = final_dataframe.loc[i, 'Price']
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / price)

final_dataframe